<H2>Diffusion Models</H2>

@article{ho2020denoising,
  title={Denoising Diffusion Probabilistic Models},
  author={Jonathan Ho and Ajay Jain and Pieter Abbeel},
  year={2020},
  journal={arXiv preprint arxiv:2006.11239}
}

Insert an image here that illustrates the model architecture.<br>
Training Instance X $\rightarrow\rightarrow\rightarrow$ Latent Instance Z<br>
Latent Instance Z $\rightarrow$ Training Instance X<br>
Randomly Generated Latent Instance Z' $\rightarrow$ New Instance X' similarly distributed as X<br>

From an applied statistics point of view, diffusion is the process of transforming a complex distribtuion to a simple one that we choose a priori. If the domain of our chosen simple distribution is the same as the domain of the original complex distribution, then under certain conditions, iterative application of a learned transition kernel on the sample of any distribution would lead to samples from the more simple distribution we choose a priori. 

Diffusion models are a class of latent variable generative models capable of generating samples that are distributed "similarly" as those in a given training dataset.<br>

Three major parts make up a diffusion model:<br>
1. forward process: incrementally add noise to a training data observation until the structure of the observation is destroyed<br>
2. reverse process: use a neural network to learn something about the forward diffusion process so that it can generate samples that are distributed similarly to the samples in the training set.<br>
3. sampling method<br>
    

$\large x_{0}\overset{forward (encoder)}{\underset{reverse (decoder)}\rightleftarrows} x_{1:T}$<br>

$\large x_{0}\overset{q(x_{1}|x_{0})}{\underset{p_{\theta}(x_{0}|x_{1})}\rightleftarrows} x_{1} \overset{q(x_{2}|x_{1})}{\underset{p_{\theta}(x_{1}|x_{2})}\rightleftarrows} x_{2}\ \dots\ x_{t} \overset{q(x_{t}|x_{t-1})}{\underset{p_{\theta}(x_{t-1}|x_{t})}\rightleftarrows}\ x_{T-1}\ \overset{q(x_{T}|x_{T-1})}{\underset{p_{\theta}(x_{T-1}|x_{T})}\rightleftarrows} x_{T}$<br>

<H3>Forward Diffusion Process Using Diagonal Gaussian</H3>

A forward diffusion process of $T\in(0,\inf)$ sequential transformations is applied to an observation of variable $x_{0}$ from the training set.<br> 

$x_{0}\rightarrow x_{1}\rightarrow\dots\rightarrow x_{T}$<br>

The joint distribution $q(x_{1:T}|x_{0})$ of latent variables $x_{1:T}$ given the original manifest variable $x_{0}$ is memoryless.<br>
$q(x_{t:T}|x_{0})=\prod\limits_{t=1}^{T}{q(x_{t}|x_{t-1})}\quad\quad\quad\quad\text{(Markov Property)}$<br>

Each latent variable $x_{t}\in x_{1:T}$ consists of the sum of noise and the value of $x_{t-1}$.<br>
Starting at point $x_{0}\sim\ q$, where $q$ is a probability distribution that will be learned, add noise to the original data in the following fashion:<br>
$x_{t}=\sqrt{1-\beta_{t}}x_{t-1}+\sqrt{\beta_{t}}z_{t}$<br>
$\text{where}$<br>
$z_{1},\dots,z_{T}\text{ are I.I.D. }\mathcal{N}(0,I)$<br> 

 Let $q(x_{t}|x_{t-1})\sim\mathcal{N}(x_{t};\sqrt{1-\beta_{t}}x_{t-1},\beta_{t}I)\quad\quad\quad\quad\quad\quad\quad$<br>
$\text{where:}\quad\quad\quad\beta_{t}\text{is the variance at time t}$<br>
$\quad\quad\quad\quad\quad\quad \beta_{1}\lt\beta_{2}\lt\dots<\beta_{T}$<br>
$\quad\quad\quad\quad\quad\quad \beta_{t}\in(0, 1)$<br>

Axiom: Any forward step of the process can be directly sampled as shown here:<br>
$q(x_{t}|x_{0})=\mathcal{N}(x_{t};\sqrt{\bar{\alpha}_{t}}x_{0},\sqrt{1-\bar{\alpha}_{t}}I)$<br>
where<br>
$\bar{\alpha}_{t}=\prod\limits_{s=1}^{t} a_{s}$<br>
and<br>
$\alpha_{s}=1-\beta_{s}$<br>


Axiom: $q(x_{T}|x_{0})\rightarrow\mathcal{N}(0,I)$ as $T\rightarrow\inf$.<br>

Let's constrain ourselves to a versy small $B_{t}\ll1$. There are claims (refs??) that using a small step size makes the reverse process of learning the diffusion process more stable.<br>

We can easily create a linear schedule of noise as used by Ho et al. in [3]. See linear-noise-scheduler.ipynb for more details. [ref to lin noise scheduler notebook]

In [1]:
import LinearNoiseScheduler as LNS

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
linearNoiseScheduler = LNS.LinearNoiseScheduler(10, 0.01, 0.2)
print("linearNoiseScheduler.betas: ", linearNoiseScheduler.betas)
print("LinearNoiseScheduler.alphas: ", linearNoiseScheduler.alphas) 


linearNoiseScheduler.betas:  tensor([0.0100, 0.0311, 0.0522, 0.0733, 0.0944, 0.1156, 0.1367, 0.1578, 0.1789,
        0.2000])
LinearNoiseScheduler.alphas:  tensor([0.9900, 0.9689, 0.9478, 0.9267, 0.9056, 0.8844, 0.8633, 0.8422, 0.8211,
        0.8000])


Ho et al. [3] show that, in addition to the reparameterization trick discussed in the section "Alternative Formulation of the ELBO Objective for Reduced Variance", sampling from any forward step using an additional auxillary noise variable $\epsilon$ may alsoe reduce variance.<br> 

$x_{t}=\sqrt{\bar{\alpha_{t}}}x_{t-1}+\sqrt{1-\bar{\alpha_{t}}}\epsilon$<br>
$\text{where}$<br>
$\epsilon\sim\mathcal{N}(0,I)\text{ and are I.I.D. }$<br> 


<H3>Reverse Diffusion Process Using Diagonal Gaussian Kernel</H3>

Now that we have discussed how the forward diffusion process takes an observation from the training data and creates a latent sample from the latent space by using incremental transformations. We will now design and train a neural network to learn the parameters needed to generate similarly distributed samples from that latent space. Transforming those generated latent samples back to the space of our training data should reveal samples that are distributed similarly as the sample observations in the training set.

According to Feller's "On the Theory of Stochastic Processes, with Particular Reference to Applications" (ref??), claim states that diffusion models can and will converge to an optimal solution.

$x_{0}\leftarrow\dots\leftarrow x_{T-1}\leftarrow x_{T}$

$p_\theta(x_{0:T})=p(x_{T})\prod\limits_{t=1}^{T}p_{\theta}(x_{t-1}|x_{t})$<br>
where<br>
$p_{\theta}(x_{T})\sim\mathcal{N}(x_{T}; 0, I)$<br>
and<br>
$p_\theta(x_{t-1}|x_{t})\sim\mathcal{N}(x_{t-1};\mu_\theta(x_{t}, t), \Sigma_\theta(x_{t}, t))$<br>


<H3>Loss Function</H3>


Here the approach of minimizing the negative log-likelihood is used as a starting point.<br>
$$\text{objective: }\underset{\theta}{\operatorname{argmin}}\left[\ -log\ p_{\theta}(x_{0})\right]$$ 

Computing the marginal distribution $p_{\theta}(x_{0})=\int p_{\theta}(x_{0:T})dx_{1:T}$ requires integration over all 1:T latent dimensions which becomes intractable quickly.<br>


One solution to the dimensionallity problem is to use a variational lower bound (also known as Evidence Lower Bound or ELBO).<br>
$$-log\ p_\theta(x_{0})\le\mathbb{E}_{x_{1:T}\sim\ q(x_{1:T}|x_{0})}\left[-\ log\frac{p_{\theta}(x_{0}|x_{1:T})}{q(x_{1:T}|x_{0})}\right]$$

$\mathbb{E}[-log\ p_{\theta}(x_{0})]\le\mathbb{E}_{q}\left[-log\frac{p_{\theta}(x_{0:T})}{q(x_{1:T}|x_{0})}\right]$<br>


$L\quad=\mathbb{E}_{q(x_{1:T}|x_{0})}\left[-\ log\ \frac{p_{\theta}(x_{0}|x_{1:T})}{q(x_{1:T}|x_{0})}\right]$<br>
$\quad\quad=\mathbb{E}_{q(x_{1:T}|x_{0})}\left[-log\ p(x_{T})-\sum\limits_{t\ge 1}log\frac{p_{\theta}(x_{t-1}|x_{t})}{q(x_{t}|x_{t-1})}\right]$<br>
$\quad\quad=\mathbb{E}_{q(x_{1:T}|x_{0})}\left[-log\ p(x_{T})-\sum\limits_{t>1}log\frac{p_{\theta}(x_{t-1}|x_{t})}{q(x_{t}|x_{t-1})}-log\frac{p_{\theta}(x_{0}|x_{1})}{q(x_{1}|x_{0})}\right]$<br>
$\quad\quad=\mathbb{E}_{q(x_{1:T}|x_{0})}\left[-log\ p(x_{T})-\sum\limits_{t>1}log\frac{p_{\theta}(x_{t-1}|x_{t})}{q(x_{t-1}|x_{t},x_{0})}\cdot\frac{q(x_{t-1}|x_{0})}{q(x_{t}|x_{0})}-log\frac{p_{\theta}(x_{0}|x_{1})}{q(x_{1}|x_{0})}\right]$<br>
$\quad\quad=\mathbb{E}_{q(x_{1:T}|x_{0})}\left[-log\ \frac{p(x_{T})}{q(x_{T}|x_{0})}-\sum\limits_{t>1}log\frac{p_{\theta}(x_{t-1}|x_{t})}{q(x_{t-1}|x_{t},x_{0})}\cdot\frac{q(x_{t-1}|x_{0})}{q(x_{t}|x_{0})}-log\ p_{\theta}(x_{0}|x_{1})\right]$<br>
$\quad\quad=\mathbb{E}_{q(x_{1:T}|x_{0})}\left[D_{KL}(q(x_{T}|x_{0})||p(x_{T}))+\sum\limits_{t>1}D_{KL}(q(x_{t-1}|x_{t},x_{0})||p_{\theta}(x_{t-1}|x_{t}))-\ log\ p_{\theta}(x_{0}|x_{1})\right]$<br>

Both of the $q(x_{t-1}|x+{t}, x_{0})$ and $p_{\theta}(x_{t-1}|x_{t}))$ arguments of the KL-Divergence term are both Gaussian and therefore the KL-Divergence term can be evaluated in closed form. This also reduces variance during the training process because instead of aiming to reconstruct Monte Carlo samples, targeting of the reverse step is the true posterior of forward process given $x_{0}$. (!insert ref here)<br>

<H3>Note on variance of ELBO using this approach</H3>

Axiom: This variational lower bound (aka the Evidence Lower Bound - ELBO) has high variance.<br>

<H4>Alternative Formulation of the ELBO Objective for Reduced Variance</H4>

Ho et al. demonstrate the derivation of a reduced objective [3] which is shown below:<br>
$L\quad=\mathbb{E}_{q(x_{1:T}|x_{0})}\left[\underbrace{D_{KL}(q(x_{T}|x_{0})||p(x_{T}))}_{L_{T}} +\sum\limits_{t>1}\underbrace{D_{KL}(q(x_{t-1}|x_{t},x_{0})||p_{\theta}(x_{t-1}|x_{t}))}_{L_{t-1}}-\ \underbrace{log\ p_{\theta}(x_{0}|x_{1})}_{L_{0}}\right]$<br>

$L_{0}\ $, Data Scaling and Reverse Process Decoder<br>
Image data consists of typical integer based RGB vectors ranging in intensity from {0 (black), 1, ..., 255} scaled linearly to [-1,1].<br>
The last term of the reverse process  is set to an independen discrete decoder derived from the Gaussian $\mathcal{N}(x_{0};\mu_{\theta}(x_{1},1),\ \sigma^{2}_{1}I)$[3]<br>
$p_{\theta}(x_{0}|x_{1})=\prod\limits_{i=1}^{D}\int_{\delta_{-}(x_{0}^{i})}^{\delta_{+}(x_{0}^{i})}\mathcal{N}(x;\mu_{\theta}^{i}(x_{1},1),\ \sigma_{1}^{2})dx$<br>

where<br>
$\delta_{+}(x)=\begin{cases}
\inf\quad\quad\quad\quad\ if\ x=1\\
x+\frac{1}{255}\quad\quad\quad\quad\ if\ x<1
\end{cases}
$<br>
and<br>
$\delta_{-}(x)=\begin{cases}
-\inf\quad\quad\quad\quad\ if\ x=-1\\
x-\frac{1}{255}\quad\quad\quad\quad\ if\ x>-1
\end{cases}
$<br>
At the end of the sampling process $\mu_{\theta}(x_{1},1)$ is displayed.<br>

$L_{T}$ and the Forward Process<br>
$\quad\ p_{\theta}(x_{T})$ is fixed since this is just the last(first) step in the forward(reverse) process.<br>
$\quad\ q(x_{t}|x_{0})$ is also fixed as was done by Ho et al. [3] (Need some rationale here still)<br>
$\quad\ $ therefore the first term of the above expectation can be simplified to:<br>
$\quad\ L\quad=\mathbb{E}_{q(x_{1:T}|x_{0})}\left[\underbrace{0}_{L_{T}} +\sum\limits_{t>1}\underbrace{D_{KL}(q(x_{t-1}|x_{t},x_{0})||p_{\theta}(x_{t-1}|x_{t}))}_{L_{t-1}}-\ \underbrace{log\ p_{\theta}(x_{0}|x_{1})}_{L_{0}}\right]$<br>


$L_{1:T-1}$ and the Reverse Process<br>
$\quad\ L_{t-1}=\mathbb{E}_{q}\left[\frac{1}{2\sigma^{2}_{t}}||\bar{\mu_{t}}(x_{t},x_{0})-\mu_{\theta}(x_{t},t)||^2\right]+C$<br>
$\quad\ $ Section 3.2 of Ho et al. [3] discuss proof of how a reparameterization of $L_{t-1}$ results in a simplified training objective shown below.<br>

$loss=\mathbb{E}_{x_{0},\ \epsilon,\ t}\left[w_{t}||\epsilon - \epsilon_{\theta}(x_{t},t)||^{2}\right]$<br>

Ho et al. [3] demonstrate that more realistic sampling can be obtained by eliminating the step-specific weighting term $w_{t}$.<br>
$loss=\mathbb{E}_{x_{0},\ \epsilon,\ t}\left[(1)||\epsilon - \epsilon_{\theta}(x_{t},t)||^{2}\right]$<br>

<H3>Modified PixelCNN++ Model Architecture</H3>

Ho et al. Describe the model architecture they used in [3].<br>
1. Their neural net architecture follows the backbone of PixelCNN++[ref PixelCNN++ paper here].<br>
2. Weight normalization [weight norm ref here] was replaced with group normalization [group norm ref here].<br>
3. Their 32x32 models use four feature map resolutions (32x32) to 4x4, and their 256x256 models use six.<br>
4. All models have two convolutional residual blocks per resolution level and self-attention blocks at the 16x16 resolution between the convolutional blocks [ref for convolutional blocks].<br>
5. Diffusion time $t$ is specified by adding the Transformer sinusoidal position embedding [ref] into each block.<br>



<H3>Training Algorithm</H3>

$\text{1.}\quad\text{repeat}$<br>
$\text{2.}\quad\quad\ x_{0}\sim\ q({x_{0}})$<br>
$\text{3.}\quad\quad\ t\sim\text{Uniform}({1,\dots,T})$<br>
$\text{4.}\quad\quad\ \epsilon\sim\mathcal{N}(0,I)$<br>
$\text{5.}\quad\quad\ \text{Take gradient descent step on}$<br>
$\quad\quad\quad\quad\nabla_{\theta}\text{||}\epsilon\ -\ \epsilon_{\theta}(\sqrt{\bar{\alpha_{t}}}x_{0} + \sqrt{1-\bar{\alpha_{t}}}\epsilon,t)\text{||}^{2}$<br>
$\text{6.}\quad\ \text{until converged.}$<br>


<H3>Sampling Algorithm</H3>

$\text{1.}\quad\ x_{T}\sim\mathcal{N}(0,I)$<br>
$\text{2.}\quad\ \text{for t=T,...,1 do}$<br>
$\text{3.}\quad\quad\ z\sim\mathcal{N}(0,I)\quad\text{if }t>1,\text{else }z=0$<br>
$\text{4.}\quad\quad\ x_{t-1}=\frac{1}{\sqrt{\alpha_{t}}}\left(x_{t}-\frac{1-\alpha_{t}}{\sqrt{1-\bar{alpha_{t}}}}\epsilon_{\theta}(x_{t},t)\right)\sigma_{t}z$<br>
$\text{5.}\quad\ \text{end for}$<br>
$\text{6.}\quad\ \text{return }x_{0}$<br>


<H3>Additional Notes On Ho et al. [3]</H3>

1. Authors conclude that a Denoising Diffusion Probabilistic Model (DDPM) can predict:<br>
    A.) mean of the noise<br>
    B.) original image<br>
    C.) noise of the image<br>
2. Authors also conclude that B is intractable.<br>
3. Authors conclude that A and C are reparamaterizations of the same problem where:<br>
4. A uses a parameterization of $\mu_{\theta}(x_{t},t)$ as a model to predict the mean of the noise added $\tilde{\mu_{t}}$<br>
5. C uses another parameterization of $\mu_{\theta}(x_{t},t)$ as a model to predict the noise added $\epsilon$.<br>

Their their respective loss functions are:<br>
1. 
2. 

Ho et al. [3] show that fixing the covariance parameter of the gaussian (shown again for convenience below) to time based constants allows us to learn the parameter for the means $\mu_{\theta}(x_{t},t)$<br>

$p_{\theta}(x_{t-1}|x_{t})\sim\mathcal{N}(x_{t-1};\mu_{\theta}(x+_{t},t),\sum_{\theta}(x_{t},t))$<br>

Diffusion is a process that satisfies a stochastic differenctial equatio and has a differential update formula of the form:<br>
$dX_{t}=\underbrace{\alpha (X_{t}, t)dt}_{\text{deterministic  component}} +\, \underbrace{(\text{random number from mean zero and  variance}\, \mu(X{t},t)dt)}_{\text{stochastic component}}.$ [4]

<H2>References</H2>

1. Sohl-Dickstein J. et al., "Deep Unsupervised Learning using
Nonequilibrium Thermodynamics" arXiv:1503.03585v, rev. Nov. 18, 2015.
2. Feller W., "On the Theory of Stochastic Processes, with Particular Reference to Applications", 
3. Ho, J. et al., "Denoising Diffuision Probabilistic Models", Dec. 16, 2020, arXi:200611239v2. 
4. "Denoising Diffusion Probabilistic Models" by ExplainingAI at www.youtube.com/watch?v=H45IF4sUgiE.